In [7]:
import os
import wrds
from dotenv import load_dotenv
import pandas as pd

load_dotenv()

db = wrds.Connection(
    wrds_username=os.getenv("WRDS_USERNAME")
)

print("WRDS connection successful")

Loading library list...
Done
WRDS connection successful


Gross Margin (매출 총 이익률) : (revtq- cogsq)/revtq
해자가 있다면 원재료 값이 올라도 판가를 올릴 수 있으므로 이 수치가 방어되거나 상승한다.

Marketing Efficiency (판관비 효율성) : saleq / xsgaq
해자가 있는 기업은 광고비 등 없어도 잘 팔린다.

ROIC (투하자본수익률) :  oiadp * (1 - txt / pret)
즉, 이자 비용을 지불하기 전 순수하게 사업으로 벌어들인 뒤 세금만 낸 돈
자본 효율성이 높다는건 적은 돈으로도 큰 수익을 낸다는 뜻이며, 이는 경쟁자가 따라올 수 없는 효율정을 가졌음을 뜻한다. 

In [8]:
fund_data = db.raw_sql("""
            SELECT 
                f.gvkey, f.datadate, f.tic AS ticker, f.cusip, f.conm,
                f.saleq, f.revtq, f.cogsq, f.xsgaq, f.oiadpq, f.txtq, f.piq, f.atq,

                (f.revtq - f.cogsq) / NULLIF(f.revtq, 0) AS gross_margin,
                (f.saleq/NULLIF(f.xsgaq, 0)) AS marketing_effi,
                (f.oiadpq * (1- f.txtq / NULLIF(f.piq,0))) / NULLIF(f.piq,0) AS ROIC
            FROM comp.fundq AS f
            INNER JOIN 
                comp.company AS c ON f.gvkey = c.gvkey
            WHERE 
                c.gind = '453010'
                AND f.datadate >= '2010-01-01'
                AND f.indfmt = 'INDL'
                AND f.datafmt = 'STD'
                AND f.popsrc = 'D'
                AND f.consol = 'C'
            """)
fund_data

,gvkey,datadate,ticker,cusip,conm,saleq,revtq,cogsq,xsgaq,oiadpq,txtq,piq,atq,gross_margin,marketing_effi,roic
0,007343,2010-02-28,MU,595112103,MICRON TECHNOLOGY INC,1961.0,1961.0,853.0,248.0,394.0,4.0,383.0,11952.0,0.565018,7.907258,1.017977
1,007772,2010-02-28,NSM.2,637640103,NATIONAL SEMICONDUCTOR CORP,361.9,361.9,95.1,150.7,93.0,19.2,72.4,2102.9,0.73722,2.40146,0.943881
2,009799,2010-02-28,SODI,834256208,SOLITRON DEVICES INC,1.976,1.976,1.399,0.329,0.201,0.014,0.204,10.233,0.292004,6.006079,0.917676
3,009999,2010-02-28,SMSC,853626109,STANDARD MICROSYSTEMS CORP,82.989,82.989,34.075,41.911,1.403,0.622,1.567,479.336,0.589403,1.980125,0.539947
4,015293,2010-02-28,FSII.1,302633102,FSI INTL INC,18.925,18.925,10.244,7.529,0.514,-0.006,0.604,66.652,0.458705,2.513614,0.859447
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9512,039454,2025-09-30,BZAI,092915107,BLAIZE HOLDINGS INC,11.867,11.867,10.091,23.997,-22.585,0.021,-26.237,60.904,0.149659,0.49452,0.861496
9513,039786,2025-09-30,VLN,M9607U115,VALENS SEMICONDUCTOR LTD,17.335,17.335,5.67,18.215,-7.293,0.014,-7.307,136.691,0.672916,0.951688,0.999996
9514,060707,2025-09-30,AMBQ,023193105,AMBIQ MICRO INC,18.165,18.165,10.049,16.405,-10.043,0.003,-8.999,186.727,0.446793,1.107284,1.116385
9515,061214,2025-09-30,ASML,N07059210,ASML HOLDING NV,8799.954,8799.954,3935.598,1652.967,2889.799,517.688,3004.781,52910.467,0.552771,5.323732,0.796038


In [3]:
price_data = db.raw_sql("""
            SELECT 
                m.ticker, d.date,
                ABS(d.prc) / NULLIF(d.cfacpr, 0) AS adj_price 
            FROM crsp.dsf AS d
            INNER JOIN crsp.msenames AS m 
                ON d.permno = m.permno
            WHERE m.namedt <= d.date AND d.date <= m.nameendt
            AND d.date >= '2010-01-01'
            AND m.ticker IN (SELECT ticker FROM comp.company WHERE gind = '453010')
""")
price_data

,ticker,date,adj_price
0,EGAS,2010-01-04,10.25
1,EGAS,2010-01-05,10.19
2,EGAS,2010-01-06,10.31
3,EGAS,2010-01-07,9.96
4,EGAS,2010-01-08,10.34
...,...,...,...
412680,TSLA,2024-12-24,462.28
412681,TSLA,2024-12-26,454.13
412682,TSLA,2024-12-27,431.66
412683,TSLA,2024-12-30,417.41


In [9]:
fund_data["datadate"] = pd.to_datetime(fund_data["datadate"])
price_data["date"] = pd.to_datetime(price_data["date"])

price_data = price_data.dropna(subset = ["adj_price"])

fund_data = fund_data.sort_values(by = "datadate")
price_data = price_data.sort_values(by = "date")

merged_data = pd.merge_asof(
    fund_data,
    price_data,
    left_on = 'datadate',
    right_on = 'date',
    by = 'ticker',
    direction = 'backward'
)

merged_data = merged_data.drop(columns = ["date"])
merged_data["rev_growth_yoy"] = merged_data.groupby('ticker')['revtq'].pct_change(periods = 4, fill_method = None)

import numpy as np
merged_data = merged_data.replace([np.inf, -np.inf], np.nan).dropna(subset = ["rev_growth_yoy", "gross_margin"])

merged_data

,gvkey,datadate,ticker,cusip,conm,saleq,revtq,cogsq,xsgaq,oiadpq,txtq,piq,atq,gross_margin,marketing_effi,roic,adj_price,rev_growth_yoy
854,001632,2011-01-31,ADI,032654105,ANALOG DEVICES INC,728.504,728.504,216.446,222.767,259.406,43.214,258.82,4596.905,0.70289,3.270251,0.834921,38.83,0.208167
856,137704,2011-01-31,OVTI,682128103,OMNIVISION TECHNOLOGIES INC,265.677,265.677,181.205,38.553,40.66,-2.608,42.11,971.763,0.31795,6.891215,1.025367,25.83,0.692911
857,117768,2011-01-31,NVDA,67066G104,NVIDIA CORP,886.376,886.376,460.017,257.188,122.778,14.255,185.906,4495.246,0.481014,3.446413,0.60979,0.598,-0.097825
858,137310,2011-01-31,MRVL,573874104,MARVELL TECHNOLOGY INC,900.513,900.513,343.538,298.307,220.464,3.345,226.198,6338.157,0.618509,3.018746,0.960237,19.01,0.068814
859,001704,2011-01-31,AMAT,038222105,APPLIED MATERIALS INC,2686.0,2686.0,1487.0,491.0,645.0,174.0,680.0,11274.0,0.446389,5.470468,0.705817,15.69,0.452754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9512,001327,2025-12-31,SWKS,83088M102,SKYWORKS SOLUTIONS INC,1035.4,1035.4,491.0,270.6,156.4,30.5,109.7,7867.9,0.525787,3.826312,1.029316,88.68,-0.030978
9513,027387,2025-12-31,PI,453204109,IMPINJ INC,92.849,92.849,41.37,50.219,-2.698,0.152,-0.987,545.186,0.554438,1.848882,3.154506,145.25999,0.013979
9514,027794,2025-12-31,WOLF,97785W106,WOLFSPEED INC,168.5,168.5,209.4,54.3,-132.6,1.2,-149.4,3445.1,-0.24273,3.103131,0.894679,6.66,-0.066482
9515,006565,2025-12-31,LRCX,512807306,LAM RESEARCH CORP,5344.791,5344.791,2589.897,840.766,1810.203,242.619,1836.613,21391.171,0.515435,6.357049,0.855419,72.23,0.221374


In [11]:
merged_data.to_csv("Moat_Analysis.csv", index=False)